# Numerical Standardizations

In [268]:
import pandas  as pd
import numpy   as np
import seaborn as sns
import time    as ti
import itertools
import matplotlib.pyplot as plt
import pickle

from pyspark.ml.classification import LogisticRegression
from pyspark.ml.linalg         import Vectors
from pyspark.ml.feature        import OneHotEncoderEstimator, StringIndexer, VectorAssembler, StandardScaler
from pyspark.ml                import Pipeline
from pyspark.mllib.stat        import Statistics

from pyspark.sql               import SparkSession, SQLContext
from pyspark.sql.types         import StructType, StructField, BooleanType, IntegerType, StringType, DoubleType, BinaryType
from pyspark.sql.functions     import countDistinct, col, desc, log, corr, lit

In [2]:
ss = SparkSession.builder\
     .config('spark.executor.memory',       '4G')\
     .config('spark.driver.memory',        '40G')\
     .config('spark.driver.maxResultSize', '10G')\
     .getOrCreate()
sc = ss.sparkContext
sq = SQLContext(sc)

# Overall Stats

In [48]:
pf = ss.read.parquet('../data/train_w261.parquet')

In [49]:
num_features = [c for c in pf.columns if 'i' in c]

In [25]:
ctr_stats = pf.describe(['ctr']).toPandas()

In [26]:
num_summary = pf.describe( num_features ).toPandas()
num_summary = pd.concat([num_summary,ctr_stats],axis=1,sort=False)

In [27]:
num_stats = num_summary.T

In [28]:
num_median = pf.approxQuantile(col=num_features,probabilities=[0.5],relativeError=0.005)
num_median.append([-999])
num_median = list(itertools.chain.from_iterable(num_median))

In [29]:
num_stats= num_stats.rename(columns=num_stats.iloc[0])

In [30]:
num_stats=num_stats.drop(num_stats.index[0])

In [31]:
num_stats['median']=num_median

In [32]:
num_stats['count'] = num_stats['count'].astype(int)

In [33]:
num_stats['mean'] = num_stats['mean'].astype(float)

In [34]:
num_stats['stddev'] = num_stats['stddev'].astype(float)

In [35]:
num_stats['median'] = num_stats['median'].astype(int)

In [36]:
num_stats['min'] = num_stats['min'].astype(float)

In [37]:
num_stats['max'] = num_stats['max'].astype(float)

In [38]:
num_stats.dtypes

count       int64
mean      float64
stddev    float64
min       float64
max       float64
median      int64
dtype: object

In [39]:
num_stats['Pearson2Skew'] = 3*(num_stats['mean']-num_stats['median'])/num_stats['stddev']

In [40]:
num_stats.loc[['ctr'],['median','Pearson2Skew']]= np.nan

In [41]:
num_stats.index.name = 'variable'
num_stats

,count,mean,stddev,min,max,median,Pearson2Skew
variable,,,,,,,
i01,20035247,3.500060,9.427662,0.0,5775.0,1.0,0.795550
i02,36673203,105.881091,391.940275,-3.0,257675.0,3.0,0.787475
i03,28802877,26.910415,396.797641,0.0,65535.0,7.0,0.150533
i04,28723574,7.322778,8.799146,0.0,969.0,4.0,1.132875
i05,35725965,18545.307976,69435.515294,0.0,23159456.0,2881.0,0.676785
i06,28469401,116.147388,391.380525,0.0,431037.0,34.0,0.629674
i07,35085776,16.322690,65.524064,0.0,34536.0,4.0,0.564191
i08,36654939,12.517513,16.816877,0.0,6047.0,8.0,0.805889
i09,35085776,106.106119,220.289905,0.0,29019.0,40.0,0.900261


In [42]:
num_stats.to_csv('num_stats.csv')

# Analysis of Each Numerical Feature

In [ ]:
print(num_features)
print(type(num_features))

In [ ]:
num_features.append('ctr')
print(num_features)


In [ ]:
idf = pf[num_features].cache()

## i01

In [ ]:
i1 = idf.select('i01','ctr').cache()

In [ ]:
smpl = i1.sample(fraction=0.000005, seed=2019)
smpl.count()

In [ ]:
smpl1=smpl.withColumn("logi01",log("i01"))
smpl1.show(30)

In [ ]:
i1_hist = i1.select('i01').rdd.flatMap(lambda x: x).histogram(5775)

In [ ]:
i1_values = i1_hist[0]
i1_values_count = i1_hist[1]

print(f"lengths: i1values={len(i1_values)}, i1counts={len(i1_values_count)}")

In [ ]:
i1_values.pop(0)
print(f"lengths: i1values={len(i1_values)}, i1counts={len(i1_values_count)}")

In [ ]:
i1_smpl = i1.sample(fraction=0.1,seed=42).toPandas()

In [ ]:
i1_smpl.describe()

In [ ]:
i1_counts = i1.groupBy('i01').count().toPandas()

In [ ]:
i1_counts.head()

In [ ]:
i1_smpl.head()

In [ ]:
fig=plt.figure(figsize=(25,5))
# Box Plot
ax=fig.add_subplot(1,5,1)
ax=sns.boxplot(x=i1_smpl['i01'])
ax.set(xlim=(-1,40))
ax.set_xlabel('Value of i01')

# Violin Plot
ax=fig.add_subplot(1,5,2)
ax=sns.violinplot(x=i1_smpl['i01'])
ax.set(xlim=(-1,40))
ax.set_xlabel('Value of i01')

# Histogram
ax=fig.add_subplot(1,5,3)
ax.hist(i1_values,len(i1_values),weights=i1_values_count,color='red')
ax.set(xlim=(-1,40))
ax.set_xlabel('Value of i01')
ax.set_ylabel('count')

# ScatterPlot
ax=fig.add_subplot(1,5,4)
ax.scatter(x=i1_counts['i01'],y=i1_counts['count'])
ax.set(xlim=(-1,40))
ax.set_xlabel('Value of i01')
ax.set_ylabel('count')

#Jointplot vs class
# ax = fig.add_subplot(1,5,5)
# ax=sns.jointplot(x=i1_smpl['i01'],y=i1_smpl['ctr'],kind='kde')
# ax.set(xlim=(-1,40))
# ax.set_xlabel('Value of i01')
# ax.set_ylabel('Value of ctr')

plt.show()

In [ ]:
i1_smpl['normed']=(i1_smpl['i01'])/(2*9.391380)
i1_smpl.describe()

In [ ]:
fig=plt.figure(figsize=(24,8))
# Box Plot
ax=fig.add_subplot(1,3,1)
ax=sns.boxplot(x=i1_smpl['normed'])
ax.set(xlim=(-0.1,2))
ax.set_xlabel('Normed Value of i01')

# Violin Plot
ax=fig.add_subplot(1,3,2)
ax=sns.violinplot(x=i1_smpl['normed'])
ax.set(xlim=(-0.1,3))
ax.set_xlabel('Normed Value of i01')

# Histogram
ax=fig.add_subplot(1,3,3)
ax.hist(i1_smpl['normed'],bins=1900)
ax.set(xlim=(-0.1,2))
ax.set_xlabel('Normed Value of i01')
ax.set_ylabel('count')

plt.show()

**Conclusion for i01**
- given the number of 0.0 values (which are NOT null), we cannot log this data, as we introduce a lot more null values
- exponentially decreasing distribution
- normalization should only consist of division by 2 times the standard deviation as we don't want to push any values down into the negative, given there are no negative values in the original data

## i02

In [ ]:
i2 = idf.select('i02').cache()

In [ ]:
negs=i2.filter(i2.i02 == -1)

In [ ]:
negs.count()

In [ ]:
i2_counts = i2.groupBy('i02').count().toPandas()

In [ ]:
i2_counts.sort_values(by=['count'],ascending=False,inplace=True)
i2_counts.head(10)

In [ ]:
smpl2 = i2.sample(fraction=0.000005, seed=2019)
smpl2.count()

In [ ]:
smpl2=smpl2.withColumn("logi02",log("i02"))
smpl2.show(30)

In [ ]:
i2_hist = i2.select('i02').rdd.flatMap(lambda x: x).histogram(20000)

In [ ]:
i2_values = i2_hist[0]
i2_values_count = i2_hist[1]

print(f"lengths: i2values={len(i2_values)}, i2counts={len(i2_values_count)}")

In [ ]:
i2_values.pop(0)
print(f"lengths: i2values={len(i2_values)}, i2counts={len(i2_values_count)}")

In [ ]:
i2_smpl = i2.sample(fraction=0.1,seed=42).toPandas()

In [ ]:
i2_smpl.describe()

In [ ]:
fig=plt.figure(figsize=(28,7))
# Box Plot
ax=fig.add_subplot(1,4,1)
ax=sns.boxplot(x=i2_smpl)
ax.set(xlim=(-20,100))
ax.set_xlabel('Value of i02')

# Violin Plot
ax=fig.add_subplot(1,4,2)
ax=sns.violinplot(x=i2_smpl)
ax.set(xlim=(-30,400))
ax.set_xlabel('Value of i02')

# Histogram
ax=fig.add_subplot(1,4,3)
ax.hist(i2_values,len(i2_values),weights=i2_values_count,color='red')
ax.set(xlim=(-20,400))
ax.set_xlabel('Value of i02')
ax.set_ylabel('count')

# ScatterPlot
ax=fig.add_subplot(1,4,4)
ax.scatter(x=i2_counts['i02'],y=i2_counts['count'])
ax.set(xlim=(-20,100))
ax.set_xlabel('Value of i02')
ax.set_ylabel('count')

plt.show()

In [ ]:
i2_smpl['normed']=(i2_smpl['i02']-3)/(391.940275)
i2_smpl.describe()

In [ ]:
i2_normed_counts = i2_counts

In [ ]:
i2_normed_counts['normed']=(i2_normed_counts['i02']-3)/(391.940275)

In [ ]:
fig=plt.figure(figsize=(28,7))
# Box Plot
ax=fig.add_subplot(1,4,1)
ax=sns.boxplot(x=i2_smpl['normed'])
ax.set(xlim=(-0.4,1))
ax.set_xlabel('Normed Value of i02')

# Violin Plot
ax=fig.add_subplot(1,4,2)
ax=sns.violinplot(x=i2_smpl['normed'])
ax.set(xlim=(-0.4,1))
ax.set_xlabel('Normed Value of i02')

# Histogram
ax=fig.add_subplot(1,4,3)
ax.hist(i2_smpl['normed'],bins=400)
ax.set(xlim=(-0.4,1))
ax.set_xlabel('Normed Value of i02')
ax.set_ylabel('count')

# ScatterPlot
ax=fig.add_subplot(1,4,4)
ax.scatter(x=i2_normed_counts['normed'],y=i2_normed_counts['count'])
ax.set(xlim=(-0.4,1))
ax.set_xlabel('Normed Value of i02')
ax.set_ylabel('count')


plt.show()

**Conclusion for i02**
- given the number of negative values (~3 million -1's) this data is a very skewed 'normal' distribution.
- with the negative numbers we will not be able to log the data
- normalization here will consist of the Z-score with median = (value-median)/StdDev.

## i03

In [ ]:
i3 = idf.select('i03').cache()

In [ ]:
i3_counts = i3.groupBy('i03').count().toPandas()

In [ ]:
i3_counts.sort_values(by=['count'],ascending=False,inplace=True)
i3_counts.head(10)

In [ ]:
i3_counts.shape

In [ ]:
zeroes = i3_counts[i3_counts['i03']==0]

In [ ]:
zeroes

In [ ]:
smpl3 = i3.sample(fraction=0.000005, seed=2019)
smpl3.count()

In [ ]:
smpl3=smpl3.withColumn("logi03",log("i03"))
smpl3.show(30)

In [ ]:
i3_hist = i3.select('i03').rdd.flatMap(lambda x: x).histogram(15000)

In [ ]:
i3_values = i3_hist[0]
i3_values_count = i3_hist[1]
i3_values.pop(0)
print(f"lengths: i3values={len(i3_values)}, i3counts={len(i3_values_count)}")

In [ ]:
i3_smpl = i3.sample(fraction=0.1,seed=42).toPandas()

In [ ]:
i3_smpl.describe()

In [ ]:
fig=plt.figure(figsize=(28,7))
# Box Plot
ax=fig.add_subplot(1,4,1)
ax=sns.boxplot(x=i3_smpl)
ax.set(xlim=(-10,200))
ax.set_xlabel('Value of i03')

# Violin Plot
ax=fig.add_subplot(1,4,2)
ax=sns.violinplot(x=i3_smpl)
ax.set(xlim=(-50,1000))
ax.set_xlabel('Value of i03')

# Histogram
ax=fig.add_subplot(1,4,3)
ax.hist(i3_values,len(i3_values),weights=i3_values_count,color='red')
ax.set(xlim=(-10,200))
ax.set_xlabel('Value of i03')
ax.set_ylabel('count')

# ScatterPlot
ax=fig.add_subplot(1,4,4)
ax.scatter(x=i3_counts['i03'],y=i3_counts['count'])
ax.set(xlim=(-10,200))
ax.set_xlabel('Value of i03')
ax.set_ylabel('count')

plt.show()

In [ ]:
i3_smpl['normed']=(i3_smpl['i03'])/(396.797641)
i3_smpl.describe()

In [ ]:
i3_normed_counts = i3_counts

In [ ]:
i3_normed_counts['normed']=(i3_normed_counts['i03'])/(396.797641)

In [ ]:
fig=plt.figure(figsize=(28,7))
# Box Plot
ax=fig.add_subplot(1,4,1)
ax=sns.boxplot(x=i3_smpl['normed'])
ax.set(xlim=(-0.02,0.75))
ax.set_xlabel('Normed Value of i03')

# Violin Plot
ax=fig.add_subplot(1,4,2)
ax=sns.violinplot(x=i3_smpl['normed'])
ax.set(xlim=(-0.2,2))
ax.set_xlabel('Normed Value of i03')

# Histogram
ax=fig.add_subplot(1,4,3)
ax.hist(i3_smpl['normed'],bins=15000)
ax.set(xlim=(-0.02,0.75))
ax.set_xlabel('Normed Value of i03')
ax.set_ylabel('count')

# ScatterPlot
ax=fig.add_subplot(1,4,4)
ax.scatter(x=i3_normed_counts['normed'],y=i3_normed_counts['count'])
ax.set(xlim=(-0.02,0.75))
ax.set_xlabel('Normed Value of i03')
ax.set_ylabel('count')


plt.show()

**Conclusion for i03**
- exponentially decreasing function
- normalization here will consist of dividing by the stddev

## i04

In [ ]:
i4 = idf.select('i04').cache()

In [ ]:
i4_counts = i4.groupBy('i04').count().toPandas()

In [ ]:
i4_counts.sort_values(by=['count'],ascending=False,inplace=True)
i4_counts.head(10)

In [ ]:
i4_counts.shape

In [ ]:
zeroes = i4_counts[i4_counts['i04']==0]

In [ ]:
zeroes

In [ ]:
i4_hist = i4.select('i04').rdd.flatMap(lambda x: x).histogram(500)

In [ ]:
i4_values = i4_hist[0]
i4_values_count = i4_hist[1]
i4_values.pop(0)
print(f"lengths: i4values={len(i4_values)}, i4counts={len(i4_values_count)}")

In [ ]:
i4_smpl = i4.sample(fraction=0.1,seed=42).toPandas()

In [ ]:
i4_smpl.describe()

In [ ]:
fig=plt.figure(figsize=(28,7))
# Box Plot
ax=fig.add_subplot(1,4,1)
ax=sns.boxplot(x=i4_smpl)
ax.set(xlim=(-1,75))
ax.set_xlabel('Value of i04')

# Violin Plot
ax=fig.add_subplot(1,4,2)
ax=sns.violinplot(x=i4_smpl)
ax.set(xlim=(-1,75))
ax.set_xlabel('Value of i04')

# Histogram
ax=fig.add_subplot(1,4,3)
ax.hist(i4_values,len(i4_values),weights=i4_values_count,color='red')
ax.set(xlim=(-1,75))
ax.set_xlabel('Value of i04')
ax.set_ylabel('count')

# ScatterPlot
ax=fig.add_subplot(1,4,4)
ax.scatter(x=i4_counts['i04'],y=i4_counts['count'])
ax.set(xlim=(-1,75))
ax.set_xlabel('Value of i04')
ax.set_ylabel('count')

plt.show()

In [ ]:
i4_smpl['normed']=(i4_smpl['i04']-4)/(8.799146)
i4_smpl.describe()

In [ ]:
i4_normed_counts = i4_counts

In [ ]:
i4_normed_counts['normed']=(i4_normed_counts['i04']-4)/(8.799146)

In [ ]:
fig=plt.figure(figsize=(28,7))
# Box Plot
ax=fig.add_subplot(1,4,1)
ax=sns.boxplot(x=i4_smpl['normed'])
ax.set(xlim=(-1,3))
ax.set_xlabel('Normed Value of i04')

# Violin Plot
ax=fig.add_subplot(1,4,2)
ax=sns.violinplot(x=i4_smpl['normed'])
ax.set(xlim=(-1,3))
ax.set_xlabel('Normed Value of i04')

# Histogram
ax=fig.add_subplot(1,4,3)
ax.hist(i4_smpl['normed'],bins=800)
ax.set(xlim=(-1,3))
ax.set_xlabel('Normed Value of i04')
ax.set_ylabel('count')

# ScatterPlot
ax=fig.add_subplot(1,4,4)
ax.scatter(x=i4_normed_counts['normed'],y=i4_normed_counts['count'])
ax.set(xlim=(-1,3))
ax.set_xlabel('Normed Value of i04')
ax.set_ylabel('count')


plt.show()

**Conclusion for i04**
- This is obviously a normal distribution (albeit skewed), given the violin plot.
- normalization here will consist of the z-score, but using median, not mean, due to the skew

## i05

In [ ]:
i5 = idf.select('i05').cache()

In [ ]:
i5_counts = i5.groupBy('i05').count().toPandas()

In [ ]:
i5_counts.sort_values(by=['count'],ascending=False,inplace=True)
i5_counts.head(15)

In [ ]:
i5_counts.shape

In [ ]:
zeroes = i5_counts[i5_counts['i05']==0]

In [ ]:
zeroes

In [ ]:
i5_hist = i5.select('i05').rdd.flatMap(lambda x: x).histogram(25000)

In [ ]:
i5_values = i5_hist[0]
i5_values_count = i5_hist[1]
i5_values.pop(0)
print(f"lengths: i5values={len(i5_values)}, i5counts={len(i5_values_count)}")

In [ ]:
i5_smpl = i5.sample(fraction=0.1,seed=42).toPandas()

In [ ]:
i5_smpl.describe()

In [ ]:
fig=plt.figure(figsize=(28,7))
# Box Plot
ax=fig.add_subplot(1,4,1)
ax=sns.boxplot(x=i5_smpl)
ax.set(xlim=(-1000,100000))
ax.set_xlabel('Value of i05')

# Violin Plot
ax=fig.add_subplot(1,4,2)
ax=sns.violinplot(x=i5_smpl)
ax.set(xlim=(-10000,100000))
ax.set_xlabel('Value of i05')

# Histogram
ax=fig.add_subplot(1,4,3)
ax.hist(i5_values,len(i5_values),weights=i5_values_count,color='red')
ax.set(xlim=(-1000,100000))
ax.set_xlabel('Value of i05')
ax.set_ylabel('count')

# ScatterPlot
ax=fig.add_subplot(1,4,4)
ax.scatter(x=i5_counts['i05'],y=i5_counts['count'])
ax.set(xlim=(-1000,100000))
ax.set_xlabel('Value of i05')
ax.set_ylabel('count')

plt.show()

In [ ]:
i5_smpl['normed']=(i5_smpl['i05']-2868)/(69435)
i5_smpl.describe()

In [ ]:
i5_normed_counts = i5_counts

In [ ]:
i5_normed_counts['normed']=(i5_normed_counts['i05']-2868)/(69435)

In [ ]:
fig=plt.figure(figsize=(28,7))
# Box Plot
ax=fig.add_subplot(1,4,1)
ax=sns.boxplot(x=i5_smpl['normed'])
ax.set(xlim=(-0.5,1.5))
ax.set_xlabel('Normed Value of i05')

# Violin Plot
ax=fig.add_subplot(1,4,2)
ax=sns.violinplot(x=i5_smpl['normed'])
ax.set(xlim=(-0.5,1.5))
ax.set_xlabel('Normed Value of i05')

# Histogram
ax=fig.add_subplot(1,4,3)
ax.hist(i5_smpl['normed'],bins=1000)
ax.set(xlim=(-0.5,1.5))
ax.set_xlabel('Normed Value of i05')
ax.set_ylabel('count')

# ScatterPlot
ax=fig.add_subplot(1,4,4)
ax.scatter(x=i5_normed_counts['normed'],y=i5_normed_counts['count'])
ax.set(xlim=(-0.5,1.5))
ax.set_xlabel('Normed Value of i05')
ax.set_ylabel('count')


plt.show()

**Conclusion for i05**
- Treating this as a skewed normal distribution, given the violin plot.
- normalization here will consist of the z-score, but using median, not mean, due to the skew

## i06

In [ ]:
i6 = idf.select('i06').cache()

In [ ]:
i6_counts = i6.groupBy('i06').count().toPandas()

In [ ]:
i6_counts.sort_values(by=['count'],ascending=False,inplace=True)
i6_counts.head(15)

In [ ]:
i6_counts.shape

In [ ]:
i6_hist = i6.select('i06').rdd.flatMap(lambda x: x).histogram(5000)

In [ ]:
i6_values = i6_hist[0]
i6_values_count = i6_hist[1]
i6_values.pop(0)
print(f"lengths: i6values={len(i6_values)}, i6counts={len(i6_values_count)}")

In [ ]:
i6_smpl = i6.sample(fraction=0.1,seed=42).toPandas()

In [ ]:
i6_smpl.describe()

In [ ]:
fig=plt.figure(figsize=(28,7))
# Box Plot
ax=fig.add_subplot(1,4,1)
ax=sns.boxplot(x=i6_smpl)
ax.set(xlim=(-100,5000))
ax.set_xlabel('Value of i06')

# Violin Plot
ax=fig.add_subplot(1,4,2)
ax=sns.violinplot(x=i6_smpl)
ax.set(xlim=(-250,5000))
ax.set_xlabel('Value of i06')

# Histogram
ax=fig.add_subplot(1,4,3)
ax.hist(i6_values,len(i6_values),weights=i6_values_count,color='red')
ax.set(xlim=(-100,5000))
ax.set_xlabel('Value of i06')
ax.set_ylabel('count')

# ScatterPlot
ax=fig.add_subplot(1,4,4)
ax.scatter(x=i6_counts['i06'],y=i6_counts['count'])
ax.set(xlim=(-100,5000))
ax.set_xlabel('Value of i06')
ax.set_ylabel('count')

plt.show()

In [ ]:
i6_smpl['normed']=(i6_smpl['i06'])/(2*391.38)
i6_smpl.describe()

In [ ]:
i6_normed_counts = i6_counts

In [ ]:
i6_normed_counts['normed']=(i6_normed_counts['i06'])/(2*391.38)

In [ ]:
fig=plt.figure(figsize=(28,7))
# Box Plot
ax=fig.add_subplot(1,4,1)
ax=sns.boxplot(x=i6_smpl['normed'])
ax.set(xlim=(-0.15,3))
ax.set_xlabel('Normed Value of i06')

# Violin Plot
ax=fig.add_subplot(1,4,2)
ax=sns.violinplot(x=i6_smpl['normed'])
ax.set(xlim=(-0.15,6))
ax.set_xlabel('Normed Value of i06')

# Histogram
ax=fig.add_subplot(1,4,3)
ax.hist(i6_smpl['normed'],bins=5000)
ax.set(xlim=(-0.15,3))
ax.set_xlabel('Normed Value of i06')
ax.set_ylabel('count')

# ScatterPlot
ax=fig.add_subplot(1,4,4)
ax.scatter(x=i6_normed_counts['normed'],y=i6_normed_counts['count'])
ax.set(xlim=(-0.15,3))
ax.set_xlabel('Normed Value of i06')
ax.set_ylabel('count')

plt.show()

**Conclusion for i06**
- Exponentially decreasing distribution
- normalization here will consist of dividing by 2 times the std. deviation.

## i07

In [ ]:
i7 = idf.select('i07').cache()

In [ ]:
i7_counts = i7.groupBy('i07').count().toPandas()

In [ ]:
i7_counts.sort_values(by=['count'],ascending=False,inplace=True)
i7_counts.head(15)

In [ ]:
i7_counts.shape

In [ ]:
i7_hist = i7.select('i07').rdd.flatMap(lambda x: x).histogram(5000)

In [ ]:
i7_values = i7_hist[0]
i7_values_count = i7_hist[1]
i7_values.pop(0)
print(f"lengths: i7values={len(i7_values)}, i7counts={len(i7_values_count)}")

In [ ]:
i7_smpl = i7.sample(fraction=0.1,seed=42).toPandas()

In [ ]:
i7_smpl.describe()

In [ ]:
fig=plt.figure(figsize=(28,7))
# Box Plot
ax=fig.add_subplot(1,4,1)
ax=sns.boxplot(x=i7_smpl)
ax.set(xlim=(-10,200))
ax.set_xlabel('Value of i07')

# Violin Plot
ax=fig.add_subplot(1,4,2)
ax=sns.violinplot(x=i7_smpl)
ax.set(xlim=(-10,200))
ax.set_xlabel('Value of i07')

# Histogram
ax=fig.add_subplot(1,4,3)
ax.hist(i7_values,len(i7_values),weights=i7_values_count,color='red')
ax.set(xlim=(-10,200))
ax.set_xlabel('Value of i07')
ax.set_ylabel('count')

# ScatterPlot
ax=fig.add_subplot(1,4,4)
ax.scatter(x=i7_counts['i07'],y=i7_counts['count'])
ax.set(xlim=(-10,200))
ax.set_xlabel('Value of i07')
ax.set_ylabel('count')

plt.show()

In [ ]:
i7_smpl['normed']=(i7_smpl['i07'])/(2*65.524)
i7_smpl.describe()

In [ ]:
i7_normed_counts = i7_counts

In [ ]:
i7_normed_counts['normed']=(i7_normed_counts['i07'])/(2*65.524)

In [ ]:
fig=plt.figure(figsize=(28,7))
# Box Plot
ax=fig.add_subplot(1,4,1)
ax=sns.boxplot(x=i7_smpl['normed'])
ax.set(xlim=(-0.1,2))
ax.set_xlabel('Normed Value of i07')

# Violin Plot
ax=fig.add_subplot(1,4,2)
ax=sns.violinplot(x=i7_smpl['normed'])
ax.set(xlim=(-0.1,3))
ax.set_xlabel('Normed Value of i07')

# Histogram
ax=fig.add_subplot(1,4,3)
ax.hist(i7_smpl['normed'],bins=2000)
ax.set(xlim=(-0.1,2))
ax.set_xlabel('Normed Value of i07')
ax.set_ylabel('count')

# ScatterPlot
ax=fig.add_subplot(1,4,4)
ax.scatter(x=i7_normed_counts['normed'],y=i7_normed_counts['count'])
ax.set(xlim=(-0.1,2))
ax.set_xlabel('Normed Value of i07')
ax.set_ylabel('count')

plt.show()

**Conclusion for i07**
- Exponentially decreasing distribution
- normalization here will consist of dividing by 2 times the std. deviation.

## i08

In [ ]:
i8 = idf.select('i08').cache()

In [ ]:
i8_counts = i8.groupBy('i08').count().toPandas()

In [ ]:
i8_counts.sort_values(by=['count'],ascending=False,inplace=True)
i8_counts.head(15)

In [ ]:
i8_counts.shape

In [ ]:
i8_hist = i8.select('i08').rdd.flatMap(lambda x: x).histogram(1200)

In [ ]:
i8_values = i8_hist[0]
i8_values_count = i8_hist[1]
i8_values.pop(0)
print(f"lengths: i8values={len(i8_values)}, i8counts={len(i8_values_count)}")

In [ ]:
i8_smpl = i8.sample(fraction=0.1,seed=42).toPandas()

In [ ]:
i8_smpl.describe()

In [ ]:
fig=plt.figure(figsize=(28,7))
# Box Plot
ax=fig.add_subplot(1,4,1)
ax=sns.boxplot(x=i8_smpl)
ax.set(xlim=(-10,200))
ax.set_xlabel('Value of i08')

# Violin Plot
ax=fig.add_subplot(1,4,2)
ax=sns.violinplot(x=i8_smpl)
ax.set(xlim=(-10,200))
ax.set_xlabel('Value of i08')

# Histogram
ax=fig.add_subplot(1,4,3)
ax.hist(i8_values,len(i8_values),weights=i8_values_count,color='red')
ax.set(xlim=(-10,200))
ax.set_xlabel('Value of i08')
ax.set_ylabel('count')

# ScatterPlot
ax=fig.add_subplot(1,4,4)
ax.scatter(x=i8_counts['i08'],y=i8_counts['count'])
ax.set(xlim=(-10,200))
ax.set_xlabel('Value of i08')
ax.set_ylabel('count')

plt.show()

In [ ]:
i8_smpl['normed']=(i8_smpl['i08'])/(2*16.816877)
i8_smpl.describe()

In [ ]:
i8_normed_counts = i8_counts

In [ ]:
i8_normed_counts['normed']=(i8_normed_counts['i08'])/(2*16.816877)

In [ ]:
fig=plt.figure(figsize=(28,7))
# Box Plot
ax=fig.add_subplot(1,4,1)
ax=sns.boxplot(x=i8_smpl['normed'])
ax.set(xlim=(-0.1,2))
ax.set_xlabel('Normed Value of i08')

# Violin Plot
ax=fig.add_subplot(1,4,2)
ax=sns.violinplot(x=i8_smpl['normed'])
ax.set(xlim=(-0.1,3))
ax.set_xlabel('Normed Value of i08')

# Histogram
ax=fig.add_subplot(1,4,3)
ax.hist(i8_smpl['normed'],bins=1200)
ax.set(xlim=(-0.1,2))
ax.set_xlabel('Normed Value of i08')
ax.set_ylabel('count')

# ScatterPlot
ax=fig.add_subplot(1,4,4)
ax.scatter(x=i8_normed_counts['normed'],y=i8_normed_counts['count'])
ax.set(xlim=(-0.1,2))
ax.set_xlabel('Normed Value of i08')
ax.set_ylabel('count')

plt.show()

**Conclusion for i08**
- Exponentially decreasing distribution
- normalization here will consist of dividing by 2 times the std. deviation.

## i09

In [ ]:
i9 = idf.select('i09').cache()

In [ ]:
i9_counts = i9.groupBy('i09').count().toPandas()

In [ ]:
i9_counts.sort_values(by=['count'],ascending=False,inplace=True)
i9_counts.head(15)

In [ ]:
i9_counts.shape

In [ ]:
i9_hist = i9.select('i09').rdd.flatMap(lambda x: x).histogram(10000)

In [ ]:
i9_values = i9_hist[0]
i9_values_count = i9_hist[1]
i9_values.pop(0)
print(f"lengths: i9values={len(i9_values)}, i9counts={len(i9_values_count)}")

In [ ]:
i9_smpl = i9.sample(fraction=0.1,seed=42).toPandas()

In [ ]:
i9_smpl.describe()

In [ ]:
fig=plt.figure(figsize=(28,7))
# Box Plot
ax=fig.add_subplot(1,4,1)
ax=sns.boxplot(x=i9_smpl)
ax.set(xlim=(-10,400))
ax.set_xlabel('Value of i09')

# Violin Plot
ax=fig.add_subplot(1,4,2)
ax=sns.violinplot(x=i9_smpl)
ax.set(xlim=(-30,500))
ax.set_xlabel('Value of i09')

# Histogram
ax=fig.add_subplot(1,4,3)
ax.hist(i9_values,len(i9_values),weights=i9_values_count,color='red')
ax.set(xlim=(-10,400))
ax.set_xlabel('Value of i09')
ax.set_ylabel('count')

# ScatterPlot
ax=fig.add_subplot(1,4,4)
ax.scatter(x=i9_counts['i09'],y=i9_counts['count'])
ax.set(xlim=(-10,400))
ax.set_xlabel('Value of i09')
ax.set_ylabel('count')

plt.show()

In [ ]:
i9_smpl['normed']=(i9_smpl['i09']-40)/(220.289905)
i9_smpl.describe()

In [ ]:
i9_normed_counts = i9_counts

In [ ]:
i9_normed_counts['normed']=(i9_normed_counts['i09']-40)/(220.289905)

In [ ]:
fig=plt.figure(figsize=(28,7))
# Box Plot
ax=fig.add_subplot(1,4,1)
ax=sns.boxplot(x=i9_smpl['normed'])
ax.set(xlim=(-.5,2))
ax.set_xlabel('Normed Value of i09')

# Violin Plot
ax=fig.add_subplot(1,4,2)
ax=sns.violinplot(x=i9_smpl['normed'])
ax.set(xlim=(-.5,2))
ax.set_xlabel('Normed Value of i09')

# Histogram
ax=fig.add_subplot(1,4,3)
ax.hist(i9_smpl['normed'],bins=10000)
ax.set(xlim=(-.5,2))
ax.set_xlabel('Normed Value of i09')
ax.set_ylabel('count')

# ScatterPlot
ax=fig.add_subplot(1,4,4)
ax.scatter(x=i9_normed_counts['normed'],y=i9_normed_counts['count'])
ax.set(xlim=(-.5,2))
ax.set_xlabel('Normed Value of i09')
ax.set_ylabel('count')

plt.show()

**Conclusion for i09**
- Violin plot: truncated skewed normal distribution.
- normalization here will consist of Z score using median instead of mean.

## i10

In [ ]:
i10 = idf.select('i10').cache()

In [ ]:
i10_counts = i10.groupBy('i10').count().toPandas()

In [ ]:
i10_counts.sort_values(by=['count'],ascending=False,inplace=True)
i10_counts.head(15)

In [ ]:
i10_counts.shape

In [ ]:
i10_hist = i10.select('i10').rdd.flatMap(lambda x: x).histogram(12)

In [ ]:
i10_values = i10_hist[0]
i10_values_count = i10_hist[1]
i10_values.pop(0)
print(f"lengths: i10values={len(i10_values)}, i10counts={len(i10_values_count)}")

In [ ]:
i10_smpl = i10.toPandas()

In [ ]:
i10_smpl.describe()

In [ ]:
fig=plt.figure(figsize=(28,7))
# Box Plot
ax=fig.add_subplot(1,4,1)
ax=sns.boxplot(x=i10_smpl)
ax.set(xlim=(-0.2,6))
ax.set_xlabel('Value of i10')

# Violin Plot
ax=fig.add_subplot(1,4,2)
ax=sns.violinplot(x=i10_smpl)
ax.set(xlim=(-0.2,6))
ax.set_xlabel('Value of i10')

# Histogram
ax=fig.add_subplot(1,4,3)
ax.hist(i10_values,len(i10_values),weights=i10_values_count,color='red')
ax.set(xlim=(-0.2,6))
ax.set_xlabel('Value of i10')
ax.set_ylabel('count')

# ScatterPlot
ax=fig.add_subplot(1,4,4)
ax.scatter(x=i10_counts['i10'],y=i10_counts['count'])
ax.set(xlim=(-0.2,6))
ax.set_xlabel('Value of i10')
ax.set_ylabel('count')

plt.show()

In [ ]:
i10_smpl['normed']=(i10_smpl['i10'])/(10)
i10_smpl.describe()

In [ ]:
i10_normed_counts = i10_counts

In [ ]:
i10_normed_counts['normed']=(i10_normed_counts['i10'])/(10)

In [ ]:
fig=plt.figure(figsize=(28,7))
# Box Plot
ax=fig.add_subplot(1,4,1)
ax=sns.boxplot(x=i10_smpl['normed'])
ax.set(xlim=(-.05,1))
ax.set_xlabel('Normed Value of i10')

# Violin Plot
ax=fig.add_subplot(1,4,2)
ax=sns.violinplot(x=i10_smpl['normed'])
ax.set(xlim=(-.05,2))
ax.set_xlabel('Normed Value of i10')

# Histogram
ax=fig.add_subplot(1,4,3)
ax.hist(i10_smpl['normed'],bins=12)
ax.set(xlim=(-.05,1))
ax.set_xlabel('Normed Value of i10')
ax.set_ylabel('count')

# ScatterPlot
ax=fig.add_subplot(1,4,4)
ax.scatter(x=i10_normed_counts['normed'],y=i10_normed_counts['count'])
ax.set(xlim=(-.05,1))
ax.set_xlabel('Normed Value of i10')
ax.set_ylabel('count')

plt.show()

**Conclusion for i10**
- almost sigmoid distribution.
- normalization here will consist of division by the maximum value only

## i11

In [ ]:
i11 = idf.select('i11').cache()

In [ ]:
i11_counts = i11.groupBy('i11').count().toPandas()

In [ ]:
i11_counts.sort_values(by=['count'],ascending=False,inplace=True)
i11_counts.head(15)

In [ ]:
i11_counts.shape

In [ ]:
i11_hist = i11.select('i11').rdd.flatMap(lambda x: x).histogram(200)

In [ ]:
i11_values = i11_hist[0]
i11_values_count = i11_hist[1]
i11_values.pop(0)
print(f"lengths: i11values={len(i11_values)}, i11counts={len(i11_values_count)}")

In [ ]:
i11_smpl = i11.sample(fraction=0.1,seed=42).toPandas()

In [ ]:
i11_smpl.describe()

In [ ]:
fig=plt.figure(figsize=(28,7))
# Box Plot
ax=fig.add_subplot(1,4,1)
ax=sns.boxplot(x=i11_smpl)
ax.set(xlim=(-1,25))
ax.set_xlabel('Value of i11')

# Violin Plot
ax=fig.add_subplot(1,4,2)
ax=sns.violinplot(x=i11_smpl)
ax.set(xlim=(-1,25))
ax.set_xlabel('Value of i11')

# Histogram
ax=fig.add_subplot(1,4,3)
ax.hist(i11_values,len(i11_values),weights=i11_values_count,color='red')
ax.set(xlim=(-1,25))
ax.set_xlabel('Value of i11')
ax.set_ylabel('count')

# ScatterPlot
ax=fig.add_subplot(1,4,4)
ax.scatter(x=i11_counts['i11'],y=i11_counts['count'])
ax.set(xlim=(-2,25))
ax.set_xlabel('Value of i11')
ax.set_ylabel('count')

plt.show()

In [ ]:
i11_smpl['normed']=(i11_smpl['i11']-1)/(5.196922)
i11_smpl.describe()

In [ ]:
i11_normed_counts = i11_counts

In [ ]:
i11_normed_counts['normed']=(i11_normed_counts['i11']-1)/(5.196922)

In [ ]:
fig=plt.figure(figsize=(28,7))
# Box Plot
ax=fig.add_subplot(1,4,1)
ax=sns.boxplot(x=i11_smpl['normed'])
ax.set(xlim=(-.5,2))
ax.set_xlabel('Normed Value of i11')

# Violin Plot
ax=fig.add_subplot(1,4,2)
ax=sns.violinplot(x=i11_smpl['normed'])
ax.set(xlim=(-.5,2))
ax.set_xlabel('Normed Value of i11')

# Histogram
ax=fig.add_subplot(1,4,3)
ax.hist(i11_smpl['normed'],bins=200)
ax.set(xlim=(-.5,2))
ax.set_xlabel('Normed Value of i11')
ax.set_ylabel('count')

# ScatterPlot
ax=fig.add_subplot(1,4,4)
ax.scatter(x=i11_normed_counts['normed'],y=i11_normed_counts['count'])
ax.set(xlim=(-.5,2))
ax.set_xlabel('Normed Value of i11')
ax.set_ylabel('count')

plt.show()

**Conclusion for i11**
- Violin plot: truncated skewed normal distribution.
- normalization here will consist of Z score using median instead of mean.

## i12

In [ ]:
i12 = idf.select('i12').cache()

In [ ]:
i12_counts = i12.groupBy('i12').count().toPandas()

In [ ]:
i12_counts.sort_values(by=['count'],ascending=False,inplace=True)
i12_counts.head(15)

In [ ]:
i12_counts.shape

In [ ]:
i12_hist = i12.select('i12').rdd.flatMap(lambda x: x).histogram(4000)

In [ ]:
i12_values = i12_hist[0]
i12_values_count = i12_hist[1]
i12_values.pop(0)
print(f"lengths: i12values={len(i12_values)}, i12counts={len(i12_values_count)}")

In [ ]:
i12_smpl = i12.sample(fraction=0.1,seed=42).toPandas()

In [ ]:
i12_smpl.describe()

In [ ]:
fig=plt.figure(figsize=(28,7))
# Box Plot
ax=fig.add_subplot(1,4,1)
ax=sns.boxplot(x=i12_smpl)
ax.set(xlim=(-0.5,10))
ax.set_xlabel('Value of i12')

# Violin Plot
ax=fig.add_subplot(1,4,2)
ax=sns.violinplot(x=i12_smpl)
ax.set(xlim=(-2,50))
ax.set_xlabel('Value of i12')

# Histogram
ax=fig.add_subplot(1,4,3)
ax.hist(i12_values,len(i12_values),weights=i12_values_count,color='red')
ax.set(xlim=(-0.5,10))
ax.set_xlabel('Value of i12')
ax.set_ylabel('count')

# ScatterPlot
ax=fig.add_subplot(1,4,4)
ax.scatter(x=i12_counts['i12'],y=i12_counts['count'])
ax.set(xlim=(-0.5,10))
ax.set_xlabel('Value of i12')
ax.set_ylabel('count')

plt.show()

In [ ]:
i12_smpl['normed']=(i12_smpl['i12'])/(2*5.672792)
i12_smpl.describe()

In [ ]:
i12_normed_counts = i12_counts

In [ ]:
i12_normed_counts['normed']=(i12_normed_counts['i12'])/(2*5.672792)

In [ ]:
fig=plt.figure(figsize=(28,7))
# Box Plot
ax=fig.add_subplot(1,4,1)
ax=sns.boxplot(x=i12_smpl['normed'])
ax.set(xlim=(-.1,1.5))
ax.set_xlabel('Normed Value of i12')

# Violin Plot
ax=fig.add_subplot(1,4,2)
ax=sns.violinplot(x=i12_smpl['normed'])
ax.set(xlim=(-.3,5))
ax.set_xlabel('Normed Value of i12')

# Histogram
ax=fig.add_subplot(1,4,3)
ax.hist(i12_smpl['normed'],bins=4000)
ax.set(xlim=(-.1,1.5))
ax.set_xlabel('Normed Value of i12')
ax.set_ylabel('count')

# ScatterPlot
ax=fig.add_subplot(1,4,4)
ax.scatter(x=i12_normed_counts['normed'],y=i12_normed_counts['count'])
ax.set(xlim=(-.1,1.5))
ax.set_xlabel('Normed Value of i12')
ax.set_ylabel('count')

plt.show()

**Conclusion for i12**
- Violin plot: exponentially decreasing distribution.
- normalization here will consist of dividing by 2 times the std. deviation.

## i13

In [ ]:
i13 = idf.select('i13').cache()

In [ ]:
i13_counts = i13.groupBy('i13').count().toPandas()

In [ ]:
i13_counts.sort_values(by=['count'],ascending=False,inplace=True)
i13_counts.head(15)

In [ ]:
i13_counts.shape

In [ ]:
i13_hist = i13.select('i13').rdd.flatMap(lambda x: x).histogram(7000)

In [ ]:
i13_values = i13_hist[0]
i13_values_count = i13_hist[1]
i13_values.pop(0)
print(f"lengths: i13values={len(i13_values)}, i13counts={len(i13_values_count)}")

In [ ]:
i13_smpl = i13.sample(fraction=0.25,seed=42).toPandas()

In [ ]:
i13_smpl.describe()

In [ ]:
fig=plt.figure(figsize=(28,7))
# Box Plot
ax=fig.add_subplot(1,4,1)
ax=sns.boxplot(x=i13_smpl)
ax.set(xlim=(-0.5,40))
ax.set_xlabel('Value of i13')

# Violin Plot
ax=fig.add_subplot(1,4,2)
ax=sns.violinplot(x=i13_smpl)
ax.set(xlim=(-10,100))
ax.set_xlabel('Value of i13')

# Histogram
ax=fig.add_subplot(1,4,3)
ax.hist(i13_values,len(i13_values),weights=i13_values_count,color='red')
ax.set(xlim=(-0.5,40))
ax.set_xlabel('Value of i13')
ax.set_ylabel('count')

# ScatterPlot
ax=fig.add_subplot(1,4,4)
ax.scatter(x=i13_counts['i13'],y=i13_counts['count'])
ax.set(xlim=(-0.5,40))
ax.set_xlabel('Value of i13')
ax.set_ylabel('count')

plt.show()

In [ ]:
i13_smpl['normed']=(i13_smpl['i13']-5)/(16.413245)
i13_smpl.describe()

In [ ]:
i13_normed_counts = i13_counts

In [ ]:
i13_normed_counts['normed']=(i13_normed_counts['i13']-5)/(16.413245)

In [ ]:
fig=plt.figure(figsize=(28,7))
# Box Plot
ax=fig.add_subplot(1,4,1)
ax=sns.boxplot(x=i13_smpl['normed'])
ax.set(xlim=(-0.5,2))
ax.set_xlabel('Normed Value of i13')

# Violin Plot
ax=fig.add_subplot(1,4,2)
ax=sns.violinplot(x=i13_smpl['normed'])
ax.set(xlim=(-0.75,5))
ax.set_xlabel('Normed Value of i13')

# Histogram
ax=fig.add_subplot(1,4,3)
ax.hist(i13_smpl['normed'],bins=7000)
ax.set(xlim=(-0.5,2))
ax.set_xlabel('Normed Value of i13')
ax.set_ylabel('count')

# ScatterPlot
ax=fig.add_subplot(1,4,4)
ax.scatter(x=i13_normed_counts['normed'],y=i13_normed_counts['count'])
ax.set(xlim=(-0.5,2))
ax.set_xlabel('Normed Value of i13')
ax.set_ylabel('count')

plt.show()

**Conclusion for i13**
- Violin plot: exponentially decreasing distribution, but the scatter plot and the bar chart look far more 'normally' distributed
- normalization here will be a Z-score, but with median instead of mean

# Summary of Numerical Variable Standardizations

In the following table, the results of the above Standardizations are shown.

In general, if the distribution of the original data appears to be exponential-decreasing, then a simple division by one or two times the standard deviation is done. If the distribution appears to be a truncated, skewed normal distribution, then a Z-score type standardization is done, however, due to the high skew on all the data, the median is used as the centering variable, not the mean. The singl variable that is sigmoid in nature is only standardized by dividing by the maximum value.

These standardizations do not attempt to transform the variables into normal distributions, but only change the values so they are all approximately in the [-1,3) range for use in machine learning.

| Numerical Variable | Distribution Type          | Standardization          |
|--------------------|----------------------------|--------------------------|
| i01                | Exponentially Decreasing   | i01' = i01/(2*SD)        |
| i02                | Truncated Skewed Normal    | i02' = (i02 - median)/SD |
| i03                | Exponentially Decreasing   | i03' = i03/SD            |
| i04                | Truncated Skewed Normal    | i04' = (i04-median)/SD   |
| i05                | Truncated Skewed Normal    | i05' = (i05-median)/SD   |
| i06                | Exponentially Decreasing   | i06' = i06/2*SD          |
| i07                | Exponentially Decreasing   | i07' = i07/2*SD          |
| i08                | Exponentially Decreasing   | i08' = i08/2*SD          |
| i09                | Truncated Skewed Normal    | i09' = (i09-median)/SD   |
| i10                | Sigmoid                    | i10' = i10/Max(i10)      |
| i11                | Truncated Skewed Normal    | i11' = (i11-median)/SD   |
| i12                | Exponentially Decreasing   | i12' = i12/2*SD          |
| i13                | Truncated Skewed Normal    | i13' = (i13-median)/SD   |

# Cross Correlation

In [ ]:
subset_idf = idf.sample(fraction=0.5,seed=42).cache()

In [ ]:
num_feat = num_features
del num_feat[-1]
num_feat

In [ ]:
# Create a correlation matrix for the numerical features, then plot
corr_mat = np.zeros([len(num_feat),len(num_feat)])
ct=0
rowno=0
for c1 in num_feat:
    rw=[]
    for c2 in num_feat:
        key='a'+str(ct)
        r = subset_idf.agg(corr(c1,c2).alias(key)).collect()[0][key]
        #print(r)
        ct +=1
        rw.append(r)
    corr_mat[rowno] = rw
    rowno +=1
    

In [ ]:
f,ax=plt.subplots(figsize=(16,16))
cmap=sns.diverging_palette(220,10,as_cmap=True)
sns.heatmap(corr_mat,cmap=cmap,annot=True)
plt.show()


# Deal with NaN values

First: are the NaN's for any given numerical variable correlated with class  
Second: How many rows would we remove if we removed the NaN rows?  
Third: If the NaNs are correlated with class, then give them an outlier value after standardization  
Fourth: If the NaNs are not correlated with class, then assign them a value = median (not mean because all the distributions are skewed).  

In [51]:
num_features.append('ctr')
print(num_features,type(num_features))

['i01', 'i02', 'i03', 'i04', 'i05', 'i06', 'i07', 'i08', 'i09', 'i10', 'i11', 'i12', 'i13', 'ctr'] <class 'list'>


In [52]:
ndf = pf.select( num_features)

In [114]:
tot_1 = ndf.filter(col('ctr')==1).count()
n_tot = ndf.count()
tot_prop_1 = tot_1/n_tot

9395606 36673203
0.25619812918986107


In [129]:
corr_mat_nan = []
ct=1
print(f'proportion of 1 in total dataset = {tot_prop_1:.3}')
for c in num_features:
    print("variable",c)
    selcols = [c,'ctr']
    icol = ndf.select(selcols).cache()
    
    # remove non-NaN rows
    icol_nans = icol.filter(col(c).isNull())
    
    # Proportion of NaNs ins variable:
    tot_nans_var = icol_nans.count()
    if tot_nans_var > 0 :
        tot_nans_var_1 = icol_nans.filter(col('ctr')==1).count()  # number of nans in class 1 for this variable
        p2 = tot_nans_var_1/tot_nans_var

        # calculate z-score
        sigmaD = np.sqrt(((tot_prop_1*(1-tot_prop_1))/n_tot)+((p2*(1-p2))/tot_nans_var))
        z = ((tot_prop_1-p2)/sigmaD)

        # find if we can reject null that the proportions are the same
        crit_z = 2.81 #99.5% two tailed
        if z > crit_z or z < -1*(crit_z):
            print(f'Proportion of NaN in class 1 for this variable, {p2:.3}, is not same as proportion of class 1 in total dataset')
        else:
            print('Proportion of NaN in class 1 for this variable is similar to proportion of 1 in total dataset. They may have a relationship. Z=',z)
    else:
        print('This variable has no NaN values')


proportion of 1 in total dataset = 0.256
variable i01
Proportion of NaN in class 1 for this variable, 0.187, is not same as proportion of class 1 in total dataset
variable i02
This variable has no NaN values
variable i03
Proportion of NaN in class 1 for this variable, 0.307, is not same as proportion of class 1 in total dataset
variable i04
Proportion of NaN in class 1 for this variable, 0.282, is not same as proportion of class 1 in total dataset
variable i05
Proportion of NaN in class 1 for this variable, 0.0975, is not same as proportion of class 1 in total dataset
variable i06
Proportion of NaN in class 1 for this variable, 0.128, is not same as proportion of class 1 in total dataset
variable i07
Proportion of NaN in class 1 for this variable, 0.143, is not same as proportion of class 1 in total dataset
variable i08
Proportion of NaN in class 1 for this variable, 0.106, is not same as proportion of class 1 in total dataset
variable i09
Proportion of NaN in class 1 for this variable

So, we can see that all the variables either have no NaNs (`i02`) or the proportion of NaNs in the variable is not the same as the proportion in the entire data set at a 99.5% confidence level. This means we either get rid of all the NaN rows or replace with Median. 

In [134]:
# Check out how many rows left if rows with NaN are dropped
total=ndf.count()
print(total)
non_NaN_df = ndf.dropna()
remaining = non_NaN_df.count()
print(remaining)
ratio_remaining = remaining/total
print(ratio_remaining)


36673203
4389727
0.11969848938474231


Only 12% of rows have all non-NaN feature values, so we must replace all NaN values with medians.

# Code to process all numerical features

In [260]:
def numerical_std(raw_train_df, train_stats):
    """Apply standardizations to all numerical features to ensure 
    numerical features have balanced weights
    
    Input: Spark Sql Dataframe of original labeled data and relevant statistics from training data
    Output: Spark Sql Dataframe of all labeled data with standardized numeric features 
    """
        
    # parse out the names of the numeric features
    num_features = [c for c in raw_train_df.columns if 'i' in c]
    
    # Replace all NaN values with the median for that variable
    medians=train_stats['medians']
    sdf = raw_train_df.fillna(medians)
    
    sds = train_stats['sds']
    maxs = train_stats['maxs']
    
    # add new stanardized columns
    sdf = sdf.withColumn('i01s',(sdf['i01']/(2.0*sds['i01SD'])))
    sdf = sdf.withColumn('i02s',((sdf['i02']-medians['i02'])/(sds['i02SD'])))
    sdf = sdf.withColumn('i03s',(sdf['i03']/(sds['i03SD'])))
    sdf = sdf.withColumn('i04s',((sdf['i04']-medians['i04'])/(sds['i04SD'])))
    sdf = sdf.withColumn('i05s',((sdf['i05']-medians['i05'])/(sds['i05SD'])))
    sdf = sdf.withColumn('i06s',(sdf['i06']/(2.0*sds['i06SD'])))
    sdf = sdf.withColumn('i07s',(sdf['i07']/(2.0*sds['i07SD'])))
    sdf = sdf.withColumn('i08s',(sdf['i08']/(2.0*sds['i08SD'])))
    sdf = sdf.withColumn('i09s',((sdf['i09']-medians['i09'])/(sds['i09SD'])))
    sdf = sdf.withColumn('i10s',(sdf['i10']/(maxs['i10Max'])))
    sdf = sdf.withColumn('i11s',((sdf['i11']-medians['i11'])/(sds['i11SD'])))
    sdf = sdf.withColumn('i12s',(sdf['i12']/(2.0*sds['i12SD'])))
    sdf = sdf.withColumn('i13s',((sdf['i13']-medians['i13'])/(sds['i13SD'])))
    
    # drop old columns
    sdf = sdf.drop('i01','i02','i04','i03','i05','i06','i07','i08','i09','i10','i11','i12','i13')
    
    # return new dataframe
    return sdf

def calc_train_stats(raw_train_df):
    """ Calculate descriptive stats on training data for use in standardization
    Input: Raw training spark sql dataframe
    Output: Dict of trainingstat:value
    """
    
    # parse out the names of the numeric features
    num_features = [c for c in raw_train_df.columns if 'i' in c]
    
    # Run describe on the numerical features, then put transposed version of results into Pandas
    num_stats = pf.describe( num_features ).toPandas().T
    
    # Calculate median, adjust indices and column names and add median to pandas dataframe
    num_median = pf.approxQuantile(col=num_features,probabilities=[0.5],relativeError=0.005)
    num_median = list(itertools.chain.from_iterable(num_median))
    num_stats= num_stats.rename(columns=num_stats.iloc[0])
    num_stats=num_stats.drop(num_stats.index[0])
    num_stats['median']=num_median
    
    # Adjust type for needed columns from object to required type
    num_stats['stddev'] = num_stats['stddev'].astype(float)
    num_stats['median'] = num_stats['median'].astype(int)
    num_stats['max'] = num_stats['max'].astype(float)
    
    # Create dictionary of required stats.
    train_stats = {
        'medians':{
            'i01': float(num_stats.loc['i01','median']),
            'i02': float(num_stats.loc['i02','median']),
            'i03': float(num_stats.loc['i03','median']),
            'i04': float(num_stats.loc['i04','median']),
            'i05': float(num_stats.loc['i05','median']),
            'i06': float(num_stats.loc['i06','median']),
            'i07': float(num_stats.loc['i07','median']),
            'i08': float(num_stats.loc['i08','median']),
            'i09': float(num_stats.loc['i09','median']),
            'i10': float(num_stats.loc['i10','median']),
            'i11': float(num_stats.loc['i11','median']),
            'i12': float(num_stats.loc['i12','median']),
            'i13': float(num_stats.loc['i13','median'])
        },
        'sds':{        
            'i01SD': num_stats.loc['i01','stddev'],
            'i02SD': num_stats.loc['i02','stddev'],
            'i03SD': num_stats.loc['i03','stddev'],
            'i04SD': num_stats.loc['i04','stddev'],
            'i05SD': num_stats.loc['i05','stddev'],
            'i06SD': num_stats.loc['i06','stddev'],
            'i07SD': num_stats.loc['i07','stddev'],
            'i08SD': num_stats.loc['i08','stddev'],
            'i09SD': num_stats.loc['i09','stddev'],
            'i11SD': num_stats.loc['i11','stddev'],
            'i12SD': num_stats.loc['i12','stddev'],
            'i13SD': num_stats.loc['i13','stddev']
        },
        'maxs':{'i10Max': num_stats.loc['i10','max']}
    }
    return(train_stats)
    

### Create Training stats and save out

In [261]:
pf = ss.read.parquet('../data/train_w261.parquet')
train_stats = calc_train_stats(pf)

In [269]:
with open('../data/train_stats.pkl','wb') as f:
    pickle.dump(train_stats,f)

### Run the standardization on the full dataset

In [270]:
%%time
standard_df= numerical_std(pf,train_stats)

CPU times: user 11.8 ms, sys: 3.42 ms, total: 15.2 ms
Wall time: 159 ms


In [273]:
%%time
standard_df.write.parquet('../data/train_w261_std.parquet')